# Advanced Lane Finding - Project 4 of Udacity's Self-Driving Car Nanodegree

1. Camera Calibration
2. Distortion correction
3. Image inspection
4. Perspective correction

## 1. Camera Calibration

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

col_count = 3
row_count = 7
fig = plt.figure(figsize=(16,32))

index = 0

# Step throuagh the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        sp = fig.add_subplot(row_count, col_count, index+1)
        plt.imshow(img)
        index += 1

plt.show()

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

## 2. Distortion Correction

In [ ]:
images = glob.glob('camera_cal/calibration*.jpg')

fig = plt.figure(figsize=(16,32))
index = 0

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    sp = fig.add_subplot(row_count, col_count, index+1)
    plt.imshow(img)
    index += 1

plt.show()

## 3. Example image inspection

In [ ]:
# Make a list of calibration images
images = glob.glob('test_images/*.jpg')
examples_images = images

col_count = 2
row_count = 8
fig = plt.figure(figsize=(20,50))

index = 0

example_image = None

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sp = fig.add_subplot(row_count, col_count, index*2+1)
    plt.imshow(img)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    sp = fig.add_subplot(row_count, col_count, index*2+2)
    plt.imshow(img)
    example_image = img
    index += 1

plt.show()

# 4. Perspective correction

In [ ]:
img_size = (example_image.shape[1],example_image.shape[0])

# defines the perspective of the camera image by defining points near the front of the car and close the the
# center of the image
relation_factor = 13.5/2.8
front_perspective_div = 3.5
back_perspective_div = front_perspective_div*relation_factor
front_perspective_y_perc = 0.92
back_perspective_y_perc = 0.65

# trapez points in order bottom left, top left, top right, bottom right (front, back, back, front)
src = [(int(img_size[0]/2-img_size[0]/front_perspective_div), int(img_size[1]*front_perspective_y_perc)), 
       (int(img_size[0]/2-img_size[0]/back_perspective_div), int(img_size[1]*back_perspective_y_perc)), 
       (int(img_size[0]/2+img_size[0]/back_perspective_div), int(img_size[1]*back_perspective_y_perc)),
       (int(img_size[0]/2+img_size[0]/front_perspective_div), int(img_size[1]*front_perspective_y_perc))]

# paint trapez into the image
image_copy = np.copy(example_image)

for index in range(4):
    pa = src[index]
    pb = src[(index+1)%4]
    cv2.line(image_copy, pa, pb, (255,0,0), 4)

fig = plt.figure(figsize=(20,20))
plt.imshow(image_copy)

margin_factor = 3

dst = [(img_size[0]//margin_factor, img_size[1]), 
       (img_size[0]//margin_factor, 0), 
       (img_size[0]-img_size[0]//margin_factor, 0), 
       (img_size[0]-img_size[0]//margin_factor, img_size[1]), 
       ]

src = np.array(src, dtype=np.float32)
dst = np.array(dst, dtype=np.float32)

tmx = cv2.getPerspectiveTransform(np.array(src), np.array(dst))

In [ ]:
width = image_copy.shape[1]
height = image_copy.shape[0]

warped = cv2.warpPerspective(example_image, tmx, (width, height))

fig = plt.figure(figsize=(20,20))
plt.imshow(warped)

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

project_video = "project_video.mp4"

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video))

### Video of the project video after distortion and perspective correction

In [ ]:
def process_image(image):
    undistorted = cv2.undistort(image, mtx, dist, None, mtx)
    warped = cv2.warpPerspective(undistorted, tmx, (width, height))
    return warped
    
from_above_video = 'test_videos_output/from_above.mp4'

white_output = from_above_video
clip1 = VideoFileClip(project_video)
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(from_above_video))